# Bird Species Classification

## Modify Data

In [17]:
# Import necessary libraries
from os.path import join, exists
from os import listdir, makedirs
from shutil import copyfile

# Define species names
species = [
    "blasti",
    "bonegl",
    "brhkyt",
    "cbrtsh",
    "cmnmyn",
    "gretit",
    "hilpig",
    "himbul",
    "himgri",
    "hsparo",
    "indvul",
    "jglowl",
    "lbicrw",
    "mgprob",
    "rebimg",
    "wcrsrt",
]

# Project directories
source_folder = "train_data/train_data/"
destination_folder = "train/"

# Function to rename the original file names from the dataset
def rename_files():
    """
    Initially, the file names are inconsistent. This function
    changes the file name to make it more understandable.

    Example - for example, DSC_6272.jpg may be changed to 100101.jpg
    For bird_specie_counter < 10, in this,
    100 -> original image, 1 -> Class Number, 01 -> Image Number

    Similarly, for the case if the species counter is greater than 10.
    """
    bird_specie_counter = 1

    for bird_specie in species:
        # Set the source image directory
        source_image_dir = join(source_folder, bird_specie)
        print(source_image_dir)
        # List source images
        source_images = listdir(source_image_dir)
        print(source_images)

        for source_image in source_images:
            # Set the destination directory
            destination = join(destination_folder, bird_specie)
            print(destination)
            if bird_specie_counter < 10:
                images = 0
                for source_image in source_images:
                    if images < 10:
                        # Copy and rename the file for species counter < 10
                        copyfile(
                            join(source_image_dir, source_image),
                            join(
                                destination,
                                str(100)
                                + str(bird_specie_counter)
                                + str(0)
                                + str(images)
                                + ".jpg",
                            ),
                        )
                    elif images >= 10:
                        # Copy and rename the file for species counter < 10
                        copyfile(
                            join(source_image_dir, source_image),
                            join(
                                destination,
                                str(100)
                                + str(bird_specie_counter)
                                + str(images)
                                + ".jpg",
                            ),
                        )
                    images += 1

            elif bird_specie_counter >= 10:
                images = 0
                for source_image in source_images:
                    if images < 10:
                        # Copy and rename the file for species counter >= 10
                        copyfile(
                            join(source_image_dir, source_image),
                            join(
                                destination,
                                str(10)
                                + str(bird_specie_counter)
                                + str(0)
                                + str(images)
                                + ".jpg",
                            ),
                        )
                    elif images >= 10:
                        # Copy and rename the file for species counter >= 10
                        copyfile(
                            join(source_image_dir, source_image),
                            join(
                                destination,
                                str(10)
                                + str(bird_specie_counter)
                                + str(images)
                                + ".jpg",
                            ),
                        )
                    images += 1

        bird_specie_counter += 1

if __name__ == "__main__":
    for bird_specie in species:
        if not exists(join(destination_folder, bird_specie)):
            # Create destination directories if they do not exist
            destination = makedirs(join(destination_folder, bird_specie))
    # Call the function to rename files
    rename_files()

train_data/train_data/blasti
['DSC_6382.jpg', 'DSC_6383.jpg', 'DSC_6384-2.jpg', 'DSC_6384.jpg', 'DSC_6385.jpg', 'DSC_6386.jpg', 'DSC_6387.jpg', 'DSC_6388.jpg', 'DSC_6389.jpg', 'DSC_6390.jpg', 'DSC_6391.jpg', 'DSC_6392.jpg', 'DSC_6393.jpg', 'DSC_6394.jpg', 'DSC_6395.jpg']
train/blasti
train/blasti
train/blasti
train/blasti
train/blasti
train/blasti
train/blasti
train/blasti
train/blasti
train/blasti
train/blasti
train/blasti
train/blasti
train/blasti
train/blasti
train_data/train_data/bonegl
['DSC_4570.jpg', 'DSC_4571.jpg', 'DSC_4583.jpg', 'DSC_4584.jpg', 'DSC_4585.jpg', 'DSC_4586.jpg']
train/bonegl
train/bonegl
train/bonegl
train/bonegl
train/bonegl
train/bonegl
train_data/train_data/brhkyt
['D72_0400.jpg', 'D72_0401.jpg', 'D72_0470.jpg', 'D72_0471.jpg', 'D72_0472.jpg']
train/brhkyt
train/brhkyt
train/brhkyt
train/brhkyt
train/brhkyt
train_data/train_data/cbrtsh
['100_5097.JPG', '_D32_10303.jpg', '_D32_10305.jpg', '_D32_10306.jpg', '_D32_10307.jpg', '_D32_10308.jpg', '_D32_10309.jpg']


## Data Augmentation

In [28]:
# Import necessary libraries
import cv2
from imgaug import augmenters as iaa
from os.path import join
import os

# Define augmentations
gauss = iaa.AdditiveGaussianNoise(scale=0.2 * 255)
blur = iaa.GaussianBlur(sigma=(3.0))
flip = iaa.Fliplr(1.0)
contrast = iaa.ContrastNormalization((0.5, 1.5), per_channel=0.5)
sharp = iaa.Sharpen(alpha=(0, 0.3), lightness=(0.7, 1.3))
affine = iaa.Affine(translate_px={"x": (-50, 50), "y": (-50, 50)})
add = iaa.Add((-20, 20), per_channel=0.5)
multiply = iaa.Multiply((0.8, 1.2), per_channel=0.5)
hue = iaa.Sequential(
    [
        iaa.ChangeColorspace(from_colorspace="RGB", to_colorspace="HSV"),
        iaa.WithChannels(0, iaa.Add((50, 100))),
        iaa.ChangeColorspace(from_colorspace="HSV", to_colorspace="RGB"),
    ]
)
aug = iaa.Sequential(
    [
        iaa.Fliplr(1.0),
        iaa.ChangeColorspace(from_colorspace="RGB", to_colorspace="HSV"),
        iaa.WithChannels(0, iaa.Add((50, 100))),
        iaa.ChangeColorspace(from_colorspace="HSV", to_colorspace="RGB"),
    ]
)

# Map species to their corresponding augmentation details
species_augmentation = {
    "blasti": [gauss, blur, flip, contrast, hue],
    "bonegl": [gauss, blur, flip, contrast, hue, add, multiply, sharp, affine],
    "brhkyt": [gauss, blur, flip, contrast, hue, add, multiply, sharp, affine],
    "cbrtsh": [gauss, blur, flip, contrast, hue, add, multiply, sharp, affine],
    "cmnmyn": [gauss, blur, flip, contrast, hue, add, multiply, sharp, affine],
    "gretit": [gauss, blur, flip, contrast, hue, add, multiply, sharp, affine], 
    "hilpig": [gauss, blur, flip, contrast, hue, add, multiply],
    "himbul": [gauss, blur, flip, contrast, hue],
    "himgri": [gauss, blur, flip, contrast, hue],
    "hsparo": [gauss, blur, flip, contrast, hue],
    "indvul": [gauss, blur, flip, contrast, hue],
    "jglowl": [gauss, blur, flip, contrast, hue, add, multiply, sharp, affine], 
    "lbicrw": [gauss, blur, flip, contrast, hue, add, multiply, sharp, affine],
    "mgprob": [gauss, blur, flip, contrast, hue, add, multiply, sharp, affine],
    "rebimg": [gauss, blur, flip, contrast, hue, add, multiply],
    "wcrsrt": [gauss, blur, flip, contrast, hue, add, multiply]
}

# Function to apply augmentations to images
def apply_augmentations(images, augmenters):
    augmented_images = []
    for aug in augmenters:
        augmented_images.extend(aug.augment_images(images))
    return augmented_images

# Loop through each species and apply augmentations
for species, augmenters in species_augmentation.items():
    # Load images for the current species
    species_folder = "./train/" + species  # Update this path according to your folder structure
    images = [cv2.imread(join(species_folder, img)) for img in os.listdir(species_folder)]

    # Apply augmentations
    augmented_images = apply_augmentations(images, augmenters)

    # Ensure the total number of images meets the required count
    total_required = 80  # Update this value based on your table
    actual = len(augmented_images) + len(images)
    if actual < total_required:
        # Apply more augmentations to match the required count
        remaining = total_required - actual
        additional_augmentations = apply_augmentations(images, augmenters)
        augmented_images.extend(additional_augmentations[:remaining])

    # Save the augmented images
    save_folder = "./augmented/" + species  # Update this path as per your folder structure
    os.makedirs(save_folder, exist_ok=True)
    for idx, img in enumerate(augmented_images):
        last_4_chars = img[-4:]
        cv2.imwrite(join(save_folder, f"20{idx}.jpg"), img)

## Create Validation Dataset

In [32]:
# Import necessary libraries
from os.path import join, exists
from os import listdir, makedirs
from shutil import move
import random

# Define species names
species = [
    "blasti",
    "bonegl",
    "brhkyt",
    "cbrtsh",
    "cmnmyn",
    "gretit",
    "hilpig",
    "himbul",
    "himgri",
    "hsparo",
    "indvul",
    "jglowl",
    "lbicrw",
    "mgprob",
    "rebimg",
    "wcrsrt",
]

# Project directories
train_dir = "./train/"
validation_dir = "./validation/"

# Separate validation data from the original train dataset based on the number of files for each species
def create_validation():
    """Validation data separation from augmented training images."""
    for bird_species in species:
        # Set paths for training and validation data
        train_imgs_path = join(train_dir, bird_species)
        destination = join(validation_dir, bird_species)

        if not exists(destination):
            makedirs(destination)

        # Get the list of training images
        train_imgs = listdir(train_imgs_path)
        number = len(train_imgs)  # Number of images in each category

        # Determine the number of images to move to validation based on the total number in the category
        if number < 78:
            validation_separation = random.sample(train_imgs, 6)
        elif 78 <= number <= 81:
            validation_separation = random.sample(train_imgs, 8)
        elif number > 85:
            validation_separation = random.sample(train_imgs, 9)

        # Move the selected images to the validation folder
        for img_file in validation_separation:
            move(join(train_imgs_path, img_file), join(destination, img_file))

if __name__ == "__main__":
    create_validation()

## Create Test and Train Dataset

In [7]:
# Import necessary libraries
import cv2
import numpy as np
from os.path import join
from os import listdir
from tensorflow.keras.utils import to_categorical

# Define species names
species = [
    "blasti",
    "bonegl",
    "brhkyt",
    "cbrtsh",
    "cmnmyn",
    "gretit",
    "hilpig",
    "himbul",
    "himgri",
    "hsparo",
    "indvul",
    "jglowl",
    "lbicrw",
    "mgprob",
    "rebimg",
    "wcrsrt",
]

# Project directories
datapath = "./"
N_CLASSES = 16  # Number of classes

def gen_data():
    """Generate numpy files for training, validation, and testing."""
    # Define numpy arrays
    X_train = []
    Y_train = []
    X_valid = []
    Y_valid = []
    X_test = []
    Y_test = []
    count = 0
    
    for bird_specie in species:
        # Samples Location
        train_data = join(datapath, "train/" + bird_specie)
        val_data = join(datapath, "validation/" + bird_specie)
        test_data = join(datapath, "test_data/test_data/" + bird_specie)

        # Samples Files
        train_files = listdir(train_data)
        valid_files = listdir(val_data)
        test_files = listdir(test_data)

        # Resize images and set color space to RGB
        for img_file in train_files:
            im = join(train_data, img_file)
            img = cv2.imread(im)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (416, 416))
            X_train.append(img)
            Y_train += [count]

        # Resize images and set color space to RGB
        for img_file in test_files:
            im = join(test_data, img_file)
            img = cv2.imread(im)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (416, 416))
            X_test.append(img)
            Y_test += [count]

        # Resize images and set color space to RGB
        for img_file in valid_files:
            im = join(val_data, img_file)
            img = cv2.imread(im)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (416, 416))
            X_valid.append(img)
            Y_valid += [count]

        count += 1
    
    # Convert to numpy array
    X_train = np.asarray(X_train).astype("float32")
    X_train /= 255
    Y_train = np.asarray(Y_train)

    # Convert to numpy array
    X_valid = np.asarray(X_valid).astype("float32")
    X_valid /= 255
    Y_valid = np.asarray(Y_valid)

    # Convert to numpy array
    X_test = np.asarray(X_test).astype("float32")
    X_test /= 255
    Y_test = np.asarray(Y_test)

    return X_valid, Y_valid

if __name__ == "__main__":
    x_valid, y_valid = gen_data()

    y_train = to_categorical(y_train, N_CLASSES)  # Convert to categorical data for target class
    y_valid = to_categorical(y_valid, N_CLASSES)  # Convert to categorical data for target class
    y_test = to_categorical(y_test, N_CLASSES)  # Convert to categorical data for target class

    np.save("X_train.npy", x_train)  # Save data as npy file
    np.save("Y_train.npy", y_train)  # Save data as npy file
    np.save("X_valid.npy", x_valid)  # Save data as npy file
    np.save("Y_valid.npy", y_valid)  # Save data as npy file
    np.save("X_test.npy", x_test)  # Save data as npy file
    np.save("Y_test.npy", y_test)  # Save data as npy file

(131, 416, 416, 3) (131, 16)


## Create Crops

In [9]:
# change directory to mask_rcnn for using mask rcnn
%cd mask_rcnn

C:\Users\iamha\OneDrive\Documents\Visual Studio Code\Python\bird_species_classification-master\mask_rcnn


In [13]:
# Assuming you're using the Mask R-CNN framework, here's a commented version of the given code:

class CocoConfig(Config):
    """
    Configuration for training on MS COCO.
    Derives from the base Config class and overrides values specific
    to the COCO dataset.
    """

    # Give the configuration a recognizable name
    NAME = "coco"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 2

    # Uncomment to train on 8 GPUs (default is 1)
    # GPU_COUNT = 8

    # Number of classes (including background)
    NUM_CLASSES = 1 + 80  # COCO has 80 classes

In [15]:
# Import necessary libraries
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import cv2
from os.path import isfile, join, exists
from os import rename, listdir, rename, makedirs

# Root directory of the project
ROOT_DIR = os.path.abspath("../train_images")

# List of bird species
species = [
    "blasti",
    "bonegl",
    "brhkyt",
    "cbrtsh",
    "cmnmyn",
    "gretit",
    "hilpig",
    "himbul",
    "himgri",
    "hsparo",
    "indvul",
    "jglowl",
    "lbicrw",
    "mgprob",
    "rebimg",
    "wcrsrt",
]

# Import Mask RCNN
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = "mask_rcnn_coco.h5"

# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")


class InferenceConfig(CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1


config = InferenceConfig()
config.display()

# Create model object in inference mode
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

# COCO Class names
class_names = [
    "BG", "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck",
    "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird",
    "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack",
    "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball",
    "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket",
    "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple",
    "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair",
    "couch", "potted plant", "bed", "dining table", "toilet", "tv", "laptop", "mouse", "remote",
    "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "book",
    "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush",
]

# Train Images path
image_path = "../train/"

# Load Test images
for bird_specie in species:
    specie = join(image_path, bird_specie)
    files = listdir(specie)
    # Files sorting
    files.sort(key=lambda f: int("".join(filter(str.isdigit, f))))

    birds = 1

    if not exists(join("../mask_rcnn_crops/", bird_specie)):
        makedirs(join("../mask_rcnn_crops/", bird_specie))
        
    # Detect birds in each class
    for file in files:
        img_path = join(specie, file)
        image = cv2.imread(img_path, 1)
        result = model.detect([image], verbose=1)
        res = result[0]
        number_of_rois = len(res["rois"])
        imgs = 1

        for j in range(number_of_rois):
            # If bird is found append then crop the bird out of that image
            if res["class_ids"][j] == 15:
                y1, x1, y2, x2 = res["rois"][j]
                crop = image[y1:y2, x1:x2]
                cv2.imwrite(
                    "../mask_rcnn_crops/"
                    + bird_specie
                    + "/"
                    + str(birds)
                    + str(imgs)
                    + ".jpg",
                    crop,
                )
                imgs += 1

        birds += 1


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                93
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

C:\Users\iamha\anaconda3\envs\mask\lib\site-packages\tensorflow\python\keras\engine\training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


Processing 1 images
image                    shape: (4000, 6016, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 6016.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (4000, 6016, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  150.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 6016.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2656, 5481, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.1

Processing 1 images
image                    shape: (4000, 6016, 3)       min:   13.00000  max:  242.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  137.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 6016.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (4000, 6016, 3)       min:   17.00000  max:  234.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  129.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 6016.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (4000, 6016, 3)       min:   16.00000  max:  254.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  148.1

Processing 1 images
image                    shape: (4000, 6016, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 6016.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2684, 6016, 3)       min:    0.00000  max:  254.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  146.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 6016.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (4000, 6016, 3)       min:   61.00000  max:  176.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:   70.1

...

Processing 1 images
image                    shape: (2160, 2880, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 2880.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2160, 2880, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 2880.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2160, 2880, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.1

Processing 1 images
image                    shape: (2160, 2880, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 2880.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2160, 2880, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 2880.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2160, 2880, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.1

Processing 1 images
image                    shape: (2160, 2880, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 2880.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2160, 2880, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 2880.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2160, 2880, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.1

Processing 1 images
image                    shape: (2160, 2880, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 2880.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2160, 2880, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 2880.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2160, 2880, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.1

Processing 1 images
image                    shape: (2160, 2880, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 2880.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2160, 2880, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 93)               min:    0.00000  max: 2880.00000  float64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
Processing 1 images
image                    shape: (2160, 2880, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.1

## Create Dataset for crops

In [16]:
# change back to orignal project directory
%cd ..

C:\Users\iamha\OneDrive\Documents\Visual Studio Code\Python\bird_species_classification-master


In [21]:
# Import necessary libraries
import cv2
import numpy as np
from os.path import join
from os import listdir
from tensorflow.keras.utils import to_categorical

# Define species names
species = [
    "blasti",
    "bonegl",
    "brhkyt",
    "cbrtsh",
    "cmnmyn",
    "gretit",
    "hilpig",
    "himbul",
    "himgri",
    "hsparo",
    "indvul",
    "jglowl",
    "lbicrw",
    "mgprob",
    "rebimg",
    "wcrsrt",
]

# Set data path and number of classes
datapath = "./"
N_CLASSES = 16  # Number of classes

def gen_data():
    """Generate numpy files for training, validation, and testing."""
    # Declare arrays
    X_train = []
    Y_train = []
    X_valid = []
    Y_valid = []
    count = 0
    for bird_specie in species:

        # Samples Location
        train_data = join(datapath, "mask_rcnn_crops/" + bird_specie)
        val_data = join(datapath, "test_crops/" + bird_specie)
        
        # Samples Files
        train_files = listdir(train_data)
        valid_files = listdir(val_data)

        # Resize images and convert to RGB color space
        for img_file in train_files:
            im = join(train_data, img_file)
            img = cv2.imread(im)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (416, 416))
            X_train.append(img)
            Y_train += [count]

        # Resize images and convert to RGB color space
        for img_file in valid_files:
            im = join(val_data, img_file)
            img = cv2.imread(im)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (416, 416))
            X_valid.append(img)
            Y_valid += [count]

        count += 1

    # Convert to numpy arrays
    X_train = np.asarray(X_train).astype("float32")
    X_train /= 255
    Y_train = np.asarray(Y_train)

    # Convert to numpy arrays
    X_valid = np.asarray(X_valid).astype("float32")
    X_valid /= 255
    Y_valid = np.asarray(Y_valid)

    return X_train, Y_train, X_valid, Y_valid

if __name__ == "__main__":
    # Generate datasets
    x_train, y_train, x_valid, y_valid = gen_data()

    # Convert to categorical data for target class
    y_train = to_categorical(y_train, N_CLASSES)
    y_valid = to_categorical(y_valid, N_CLASSES)

    # Print dataset shapes
    print(x_valid.shape, y_valid.shape)
    print(x_train.shape, y_train.shape)

    # Save datasets as npy files
    np.save("X_train_cropped.npy", x_train)
    np.save("Y_train_cropped.npy", y_train)
    np.save("X_valid_cropped.npy", x_valid)
    np.save("Y_valid_cropped.npy", y_valid)

(116, 416, 416, 3) (116, 16)
(2207, 416, 416, 3) (2207, 16)


## Inception V3

In [8]:
import math
import numpy as np
from keras.layers import (
    Dense,
    Activation,
    Dropout,
    Flatten,
    AveragePooling2D,
)
from keras.optimizers import Adam
from keras.models import Model
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from tensorflow.keras.saving import get_custom_objects
from keras.applications.inception_v3 import InceptionV3


BATCH_SIZE = 32
VALIDATION_SPLIT = 0.1
N_CLASSES = 16
EPOCHS = 7


def swish(x):
    return K.sigmoid(x) * x

# get swish activation function
get_custom_objects().update({"swish": Activation(swish)})


# Learning Step Decay by 10e-1 after every 4 epochs
def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.1
    epochs_drop = 4.0
    lrate = initial_lrate * math.pow(drop, math.floor((epoch) / epochs_drop))
    return lrate


# Calculates Precision Accuracy
def precision(y_true, y_pred):
    """Precision metric.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


# Calculates Recall Accuracy
def recall(y_true, y_pred):
    """Recall metric.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


# Calculates F1 score
def f1(y_true, y_pred):
    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))


# Inecption_V3 model define
def build_inceptionV3(
    img_shape=(416, 416, 3),
    n_classes=16,
    l2_reg=0.0,
    load_pretrained=True,
    freeze_layers_from="base_model",
):
    # Decide if load pretrained weights from imagenet
    if load_pretrained:
        weights = "imagenet"
    else:
        weights = None

    # Get base model
    base_model = InceptionV3(
        include_top=False,
        weights=weights,
        input_tensor=None,
        input_shape=img_shape
    )

    # Add final layers
    x = base_model.output
    x = AveragePooling2D((8, 8), strides=(8, 8), name="avg_pool")(x)
    x = Flatten(name="flatten")(x)
    x = Dense(
              512,
              activation="swish",
              name="dense_1",
              kernel_initializer="he_uniform"
    )(x)
    x = Dropout(0.25)(x)
    predictions = Dense(
        n_classes,
        activation="softmax",
        name="predictions",
        kernel_initializer="he_uniform",
    )(x)

    # This is the model to be trained
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze some layers
    if freeze_layers_from is not None:
        if freeze_layers_from == "base_model":
            print("   Freezing base model layers")
            for layer in base_model.layers:
                layer.trainable = False
        else:
            for i, layer in enumerate(model.layers):
                print(i, layer.name)
            print("   Freezing from layer 0 to " + str(freeze_layers_from))
            for layer in model.layers[:freeze_layers_from]:
                layer.trainable = False
            for layer in model.layers[freeze_layers_from:]:
                layer.trainable = True

    # Compiling Model with Adam Optimizer
    adam = Adam(0.0001)
    model.compile(
        loss="categorical_crossentropy",
        optimizer=adam,
        metrics=[precision, recall, f1]
    )
    return model


# Loading Cropped Images for Training resized to 416x416
x_train_crop = np.load('X_train_cropped.npy')
y_train_crop = np.load('Y_train_cropped.npy')

# Loading Original Images for training resized to 416x416
x_train_original = np.load('X_train.npy')
y_train_original = np.load('Y_train.npy')
x_valid          = np.load('X_valid.npy')
y_valid          = np.load('Y_valid.npy')

# Loading Original Images for Testing resized to 416x416
x_test = np.load("X_test.npy")
y_test = np.load("Y_test.npy")

print(x_valid.shape, y_valid.shape)

# Learning Rate Schedule
lrate = LearningRateScheduler(step_decay)

# Loading Model
model = build_inceptionV3()

# Loading Trained weights
# model.load_weights("inception_v3_model.h5")

# Model Fitting
history = model.fit(x_train_original, y_train_original,
      batch_size=BATCH_SIZE,
      epochs=EPOCHS,
      verbose= 1,
    # steps_per_epoch=x_train.shape[0]//BATCH_SIZE,
    callbacks = [lrate],
    validation_data=(x_valid, y_valid)
    )

# Save model weights
model.save_weights('inception_v3_crops+images.h5')

# Calculate score over test data
score = model.evaluate(x_test, y_test, verbose=1, batch_size=BATCH_SIZE)

# Prints Precision, Recall, and F-1 score
print(score)

(131, 416, 416, 3) (131, 16)
   Freezing base model layers
Epoch 1/7
39/39 [==============================] - 571s 14s/step - loss: 0.9857 - precision: 0.8008 - recall: 0.5542 - f1: 0.6295 - val_loss: 0.3786 - val_precision: 0.8710 - val_recall: 0.7833 - val_f1: 0.8234 - lr: 0.0010
Epoch 2/7
39/39 [==============================] - 557s 14s/step - loss: 0.1694 - precision: 0.9699 - recall: 0.9375 - f1: 0.9530 - val_loss: 0.1452 - val_precision: 0.9075 - val_recall: 0.8958 - val_f1: 0.9016 - lr: 0.0010
Epoch 3/7
39/39 [==============================] - 502s 13s/step - loss: 0.0644 - precision: 0.9877 - recall: 0.9776 - f1: 0.9826 - val_loss: 0.1125 - val_precision: 0.9144 - val_recall: 0.9083 - val_f1: 0.9113 - lr: 0.0010
Epoch 4/7
39/39 [==============================] - 544s 14s/step - loss: 0.0443 - precision: 0.9920 - recall: 0.9888 - f1: 0.9903 - val_loss: 0.0873 - val_precision: 0.9871 - val_recall: 0.9812 - val_f1: 0.9841 - lr: 0.0010
Epoch 5/7
39/39 [============================

In [1]:
import math
import numpy as np
from keras.layers import (
    Dense,
    Activation,
    Dropout,
    Flatten,
    AveragePooling2D,
)
from keras.optimizers import Adam
from keras.models import Model
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from tensorflow.keras.saving import get_custom_objects
from keras.applications.inception_v3 import InceptionV3


BATCH_SIZE = 32
VALIDATION_SPLIT = 0.1
N_CLASSES = 16
EPOCHS = 7


def swish(x):
    return K.sigmoid(x) * x


get_custom_objects().update({"swish": Activation(swish)})


# Learning Step Decay by 10e-1 after every 4 epochs
def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.1
    epochs_drop = 4.0
    lrate = initial_lrate * math.pow(drop, math.floor((epoch) / epochs_drop))
    return lrate


# Calculates Precision Accuracy
def precision(y_true, y_pred):
    """Precision metric.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


# Calculates Recall Accuracy
def recall(y_true, y_pred):
    """Recall metric.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


# Calculates F1 score
def f1(y_true, y_pred):
    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))


# Inecption_V3 model define
def build_inceptionV3(
    img_shape=(416, 416, 3),
    n_classes=16,
    l2_reg=0.0,
    load_pretrained=True,
    freeze_layers_from="base_model",
):
    # Decide if load pretrained weights from imagenet
    if load_pretrained:
        weights = "imagenet"
    else:
        weights = None

    # Get base model
    base_model = InceptionV3(
        include_top=False,
        weights=weights,
        input_tensor=None,
        input_shape=img_shape
    )

    # Add final layers
    x = base_model.output
    x = AveragePooling2D((8, 8), strides=(8, 8), name="avg_pool")(x)
    x = Flatten(name="flatten")(x)
    x = Dense(
              512,
              activation="swish",
              name="dense_1",
              kernel_initializer="he_uniform"
    )(x)
    x = Dropout(0.25)(x)
    predictions = Dense(
        n_classes,
        activation="softmax",
        name="predictions",
        kernel_initializer="he_uniform",
    )(x)

    # This is the model to be trained
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze some layers
    if freeze_layers_from is not None:
        if freeze_layers_from == "base_model":
            print("   Freezing base model layers")
            for layer in base_model.layers:
                layer.trainable = False
        else:
            for i, layer in enumerate(model.layers):
                print(i, layer.name)
            print("   Freezing from layer 0 to " + str(freeze_layers_from))
            for layer in model.layers[:freeze_layers_from]:
                layer.trainable = False
            for layer in model.layers[freeze_layers_from:]:
                layer.trainable = True

    # Compiling Model with Adam Optimizer
    adam = Adam(0.0001)
    model.compile(
        loss="categorical_crossentropy",
        optimizer=adam,
        metrics=[precision, recall, f1]
    )
    return model


# Loading Cropped Images for Training resized to 416x416
x_train_crop = np.load('X_train_cropped.npy')
y_train_crop = np.load('Y_train_cropped.npy')

# Loading Original Images for training resized to 416x416
# x_train_original = np.load('X_train.npy')
# y_train_original = np.load('Y_train.npy')
x_valid          = np.load('X_valid_cropped.npy')
y_valid          = np.load('Y_valid_cropped.npy')

# Loading Original Images for Testing resized to 416x416
x_test = np.load("X_test.npy")
y_test = np.load("Y_test.npy")

print(x_valid.shape, y_valid.shape)

# Learning Rate Schedule
lrate = LearningRateScheduler(step_decay)

# Loading Model
model = build_inceptionV3()

# Loading Trained weights
# model.load_weights("inception_v3_model.h5")

# Model Fitting
history = model.fit(x_train_crop, y_train_crop,
      batch_size=BATCH_SIZE,
      epochs=EPOCHS,
      verbose= 1,
    # steps_per_epoch=x_train.shape[0]//BATCH_SIZE,
    callbacks = [lrate],
    validation_data=(x_valid, y_valid)
    )

# Save model weights
model.save_weights('inception_v3_crops.h5')

# Calculate score over test data
score = model.evaluate(x_test, y_test, verbose=1, batch_size=BATCH_SIZE)

# Prints Precision, Recall, and F-1 score
print(score)



(116, 416, 416, 3) (116, 16)

   Freezing base model layers
Epoch 1/7


69/69 [==============================] - 1225s 18s/step - loss: 0.8511 - precision: 0.8903 - recall: 0.6679 - f1: 0.7525 - val_loss: 0.7711 - val_precision: 0.8562 - val_recall: 0.6750 - val_f1: 0.7505 - lr: 0.0010
Epoch 2/7
69/69 [==============================] - 1196s 17s/step - loss: 0.2200 - precision: 0.9578 - recall: 0.9089 - f1: 0.9323 - val_loss: 0.5410 - val_precision: 0.9030 - val_recall: 0.8156 - val_f1: 0.8557 - lr: 0.0010
Epoch 3/7
69/69 [==============================] - 1223s 18s/step - loss: 0.0943 - precision: 0.9815 - recall: 0.9669 - f1: 0.9741 - val_loss: 0.4385 - val_precision: 0.9155 - val_recall: 0.8891 - val_f1: 0.9020 - lr: 0.0010
Epoch 4/7
69/69 [==============================] - 1361s 20s/step - loss: 0.0747 - precision: 0.9835 - recall: 0.9706 - f1: 0.9769 - val_loss: 0.5086 - val_precision: 0.8878 - val_recall: 0.8813 - val_f1: 0.8845 - lr: 0.0010
Epoch 5/7
69/69 [===================

## Inception Resnet V2

In [2]:
import math
import numpy as np
from keras.layers import (
    Dense,
    Activation,
    Dropout,
    Flatten,
    AveragePooling2D,
)
from keras.optimizers import Adam
from keras.models import Model
from keras.callbacks import LearningRateScheduler
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras import backend as K
from tensorflow.keras.saving import get_custom_objects


BATCH_SIZE = 32
VALIDATION_SPLIT = 0.1
N_CLASSES = 16
EPOCHS = 7


# Swish Activation Function
def swish(x):
    return K.sigmoid(x) * x


get_custom_objects().update({"swish": Activation(swish)})


# Learning Step Decay by 10e-1 after every 4 epochs
def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.1
    epochs_drop = 4.0
    lrate = initial_lrate * math.pow(drop, math.floor((epoch) / epochs_drop))
    return lrate


# Calculates Precision Accuracy
def precision(y_true, y_pred):
    """Precision metric.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


# Calculates Recall Accuracy
def recall(y_true, y_pred):
    """Recall metric.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


# Calculates F1 score
def f1(y_true, y_pred):
    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))


# Inception_ResNet_V2 model define
def build_inception_resnet_V2(
    img_shape=(416, 416, 3),
    n_classes=16,
    l2_reg=0.0,
    load_pretrained=True,
    freeze_layers_from="base_model",
):

    # Decide if load pretrained weights from imagenet
    if load_pretrained:
        weights = "imagenet"
    else:
        weights = None

    # Get base model
    base_model = InceptionResNetV2(
        include_top=False,
        weights=weights,
        input_tensor=None,
        input_shape=img_shape
    )

    # Add final layers
    x = base_model.output
    x = AveragePooling2D((8, 8), strides=(8, 8), name="avg_pool")(x)
    x = Flatten(name="flatten")(x)
    x = Dense(
              512,
              activation="swish",
              name="dense_1",
              kernel_initializer="he_uniform"
    )(x)
    x = Dropout(0.25)(x)
    predictions = Dense(
        n_classes,
        activation="softmax",
        name="predictions",
        kernel_initializer="he_uniform",
    )(x)

    # This is the model that will  be trained
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze some layers
    if freeze_layers_from is not None:
        if freeze_layers_from == "base_model":
            print("   Freezing base model layers")
            for layer in base_model.layers:
                layer.trainable = False
        else:
            for i, layer in enumerate(model.layers):
                print(i, layer.name)
            print("   Freezing from layer 0 to " + str(freeze_layers_from))
            for layer in model.layers[:freeze_layers_from]:
                layer.trainable = False
            for layer in model.layers[freeze_layers_from:]:
                layer.trainable = True

    # Compiling Model with Adam Optimizer
    adam = Adam(0.0001)
    model.compile(
        loss="categorical_crossentropy",
        optimizer=adam,
        metrics=[precision, recall, f1]
    )
    return model

# Loading Cropped Images for Training resized to 416x416
# x_train_crop = np.load('X_train_cropped.npy')
# y_train_crop = np.load('Y_train_cropped.npy')

# Loading Original Images for training resized to 416x416
x_train_original = np.load('X_train.npy')
y_train_original = np.load('Y_train.npy')
x_valid          = np.load('X_valid.npy')
y_valid          = np.load('Y_valid.npy')

# # Loading Original Images for Testing rsized to 416x416
# x_test = np.load("X_test.npy")
# y_test = np.load("Y_test_categorical.npy")

# print(x_train.shape, y_train.shape)

# Learning Rate Schedule
lrate = LearningRateScheduler(step_decay)

# Loading Model
model = build_inception_resnet_V2()

# Loading Trained weights
# model.load_weights("inception_resnet_v2_images+crops.h5")

# Model Fitting with 10% of the images used for Validation purpose
history = model.fit(x_train_original, y_train_original,
      batch_size=BATCH_SIZE,
      epochs=EPOCHS,
      verbose= 1,
    # steps_per_epoch=x_train.shape[0]//BATCH_SIZE,
    callbacks = [lrate],
    validation_data=(x_valid, y_valid)
    )

# Save Model Weights
model.save_weights('inception_resnet_crops.h5')

# Calculate score over test data
score = model.evaluate(x_test, y_test, verbose=1, batch_size=BATCH_SIZE)

# Prints Precision, Recall, and F-1 score
print(score)

   Freezing base model layers
Epoch 1/7


39/39 [==============================] - 1006s 26s/step - loss: 1.0678 - precision: 0.8092 - recall: 0.5446 - f1: 0.6230 - val_loss: 0.4513 - val_precision: 0.9364 - val_recall: 0.8313 - val_f1: 0.8791 - lr: 0.0010
Epoch 2/7
39/39 [==============================] - 702s 18s/step - loss: 0.2403 - precision: 0.9461 - recall: 0.9014 - f1: 0.9229 - val_loss: 0.2922 - val_precision: 0.9546 - val_recall: 0.9250 - val_f1: 0.9393 - lr: 0.0010
Epoch 3/7
39/39 [==============================] - 534s 14s/step - loss: 0.1642 - precision: 0.9515 - recall: 0.9295 - f1: 0.9402 - val_loss: 0.2452 - val_precision: 0.9423 - val_recall: 0.9250 - val_f1: 0.9334 - lr: 0.0010
Epoch 4/7
39/39 [==============================] - 519s 13s/step - loss: 0.1000 - precision: 0.9787 - recall: 0.9591 - f1: 0.9687 - val_loss: 0.1909 - val_precision: 0.9604 - val_recall: 0.9312 - val_f1: 0.9453 - lr: 0.0010
Epoch 5/7
39/39 [==============================] - 535s 14s/step - loss

NameError: name 'x_test' is not defined

In [3]:
x_test = np.load("X_test.npy")
y_test = np.load("Y_test.npy")

score = model.evaluate(x_test, y_test, verbose=1, batch_size=BATCH_SIZE)

# Prints Precision, Recall, and F-1 score
print(score)

5/5 [==============================] - 232s 47s/step - loss: 1.8820 - precision: 0.6747 - recall: 0.6028 - f1: 0.6359
[1.8820019960403442, 0.6747381687164307, 0.6028016805648804, 0.6359087228775024]


## Inception ResNet V2 Cropped

In [19]:
%cd ..

C:\Users\iamha\OneDrive\Documents\Visual Studio Code\Python\bird_species_classification-master


In [2]:
import math
import numpy as np
from keras.layers import (
    Dense,
    Activation,
    Dropout,
    Flatten,
    AveragePooling2D,
)
from keras.optimizers import Adam
from keras.models import Model
from keras.callbacks import LearningRateScheduler
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras import backend as K
from tensorflow.keras.saving import get_custom_objects


BATCH_SIZE = 32
VALIDATION_SPLIT = 0.1
N_CLASSES = 16
EPOCHS = 7


# Swish Activation Function
def swish(x):
    return K.sigmoid(x) * x


get_custom_objects().update({"swish": Activation(swish)})


# Learning Step Decay by 10e-1 after every 4 epochs
def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.1
    epochs_drop = 4.0
    lrate = initial_lrate * math.pow(drop, math.floor((epoch) / epochs_drop))
    return lrate


# Calculates Precision Accuracy
def precision(y_true, y_pred):
    """Precision metric.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


# Calculates Recall Accuracy
def recall(y_true, y_pred):
    """Recall metric.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


# Calculates F1 score
def f1(y_true, y_pred):
    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))


# Inception_ResNet_V2 model define
def build_inception_resnet_V2(
    img_shape=(416, 416, 3),
    n_classes=16,
    l2_reg=0.0,
    load_pretrained=True,
    freeze_layers_from="base_model",
):

    # Decide if load pretrained weights from imagenet
    if load_pretrained:
        weights = "imagenet"
    else:
        weights = None

    # Get base model
    base_model = InceptionResNetV2(
        include_top=False,
        weights=weights,
        input_tensor=None,
        input_shape=img_shape
    )

    # Add final layers
    x = base_model.output
    x = AveragePooling2D((8, 8), strides=(8, 8), name="avg_pool")(x)
    x = Flatten(name="flatten")(x)
    x = Dense(
              512,
              activation="swish",
              name="dense_1",
              kernel_initializer="he_uniform"
    )(x)
    x = Dropout(0.25)(x)
    predictions = Dense(
        n_classes,
        activation="softmax",
        name="predictions",
        kernel_initializer="he_uniform",
    )(x)

    # This is the model that will  be trained
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze some layers
    if freeze_layers_from is not None:
        if freeze_layers_from == "base_model":
            print("   Freezing base model layers")
            for layer in base_model.layers:
                layer.trainable = False
        else:
            for i, layer in enumerate(model.layers):
                print(i, layer.name)
            print("   Freezing from layer 0 to " + str(freeze_layers_from))
            for layer in model.layers[:freeze_layers_from]:
                layer.trainable = False
            for layer in model.layers[freeze_layers_from:]:
                layer.trainable = True

    # Compiling Model with Adam Optimizer
    adam = Adam(0.0001)
    model.compile(
        loss="categorical_crossentropy",
        optimizer=adam,
        metrics=[precision, recall, f1]
    )
    return model

# Loading Cropped Images for Training resized to 416x416
# x_train_crop = np.load('X_train_cropped.npy')
# y_train_crop = np.load('Y_train_cropped.npy')

# Loading Original Images for training resized to 416x416
# x_train_original = np.load('X_train.npy')
# y_train_original = np.load('Y_train.npy')
# x_valid          = np.load('X_valid_cropped.npy')
# y_valid          = np.load('Y_valid_cropped.npy')

# # Loading Original Images for Testing rsized to 416x416
x_test = np.load("X_test.npy")
y_test = np.load("Y_test.npy")

# print(x_train.shape, y_train.shape)

# Learning Rate Schedule
lrate = LearningRateScheduler(step_decay)

# Loading Model
model = build_inception_resnet_V2()

# Loading Trained weights
model.load_weights("inception_resnet_images+crops.h5")

# Model Fitting with 10% of the images used for Validation purpose
# history = model.fit(x_train_crop, y_train_crop,
#       batch_size=BATCH_SIZE,
#       epochs=EPOCHS,
#       verbose= 1,
#     # steps_per_epoch=x_train.shape[0]//BATCH_SIZE,
#     callbacks = [lrate],
#     validation_data=(x_valid, y_valid)
#     )

# Save Model Weights
# model.save_weights('inception_resnet_images+crops.h5')

# Calculate score over test data
score = model.evaluate(x_test, y_test, verbose=1, batch_size=BATCH_SIZE)

# Prints Precision, Recall, and F-1 score
print(score)

   Freezing base model layers


5/5 [==============================] - 254s 49s/step - loss: 2.0527 - precision: 0.6042 - recall: 0.5416 - f1: 0.5709
[2.0526747703552246, 0.6041890382766724, 0.5415948629379272, 0.570874810218811]
